In [17]:
import cmocean as cmo
import gc
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
# # Load predictions
# fnam = "/home/jbassham/jack/thesis-rough/data/sh/outputs/cnn/weighted/CNNPreds_sh_1992_2020.npz"
# data = np.load(fnam, allow_pickle = True)

# ypred = data['y_pred']
# ytrue = data['y_true']

In [ ]:
# Load predictions
fnam = "/home/jbassham/jack/thesis-rough/data/sh/outputs/cnn/weighted/CNNPreds_sh_1992_2020.npz"
data = np.load(fnam, allow_pickle = True)

upred = data['y_pred'][:,0,:,:]
vpred = data['y_pred'][:,1,:,:]

utrue = data['y_true'][:,0,:,:]
vtrue = data['y_true'][:,1,:,:]

In [4]:
# Delete unused variables from memory
del data
gc.collect()

44

In [5]:
# load latitude longitude grid for plotting
fnam = "/home/jbassham/jack/thesis-rough/data/sh/inputs/lat_lon_sh_1992_2020.npz"
data = np.load(fnam, allow_pickle = True)

lat = data['lat']
lon = data['lon']

In [6]:
# Load CNN data split indices and land mask
fnam = "/home/jbassham/jack/thesis-rough/data/sh/cnn-inputs/indices_land_sh_1992_2020.npz"
data = np.load(fnam, allow_pickle = True)

train_idx = data['train_idx']
val_idx = data['val_idx']
test_idx = data['test_idx']
land_mask = data['land_mask']

In [7]:
# Load uncertainty, ice velocities
fnam = "/home/jbassham/jack/thesis-rough/data/sh/inputs/inputs_normalized_sh_1992_2020.npz"
data = np.load(fnam, allow_pickle=True)
rt = data['rtn']
uit = data['uitn']
vit = data['vitn']

In [8]:
# Crop uncertainty, ice velocities to test indices
r_test = rt[test_idx,:,:]
ui_test = uit[test_idx,:,:] 
vi_test = vit[test_idx,:,:]

In [9]:
# Delete unused variables from memory
del data, rt, uit, vit
gc.collect()

99

In [10]:
def correlation(pred, true):

    """
    Pearson Correlation
    """

    predbar = np.nanmean(pred, axis = 0) # mean predicted
    truebar = np.nanmean(true, axis = 0) # mean true

    covariance = np.nansum((pred - predbar) * (true - truebar), axis = 0) # covariance between predicted and true
    
    stdpred = np.sqrt(np.nansum((pred - predbar)**2), axis = 0) # standard deviation predited
    stdtrue = np.sqrt(np.nansum((true - truebar)**2), axis = 0) # standard deviation true

    correlation = covariance / (stdpred * stdtrue)

    return correlation

In [11]:
def weighted_correlation(pred, true, r, epsilon = 1e-4):

    """
    Weighted Pearson Correlation referenced from:
    https://www.air.org/sites/default/files/2021-06/Weighted-and-Unweighted-Correlation-Methods-Large-Scale-Educational-Assessment-April-2018.pdf
    
    """

    w = 1 / (r + epsilon)

    def weighted_mean(x, w):
        return np.nansum(w * w, axis = 0) / np.nansum(w, axis = 0)

    predbar = weighted_mean(pred, axis = 0) # weighted mean predicted
    truebar = weighted_mean(true, axis = 0) # weighted mean true

    weighted_cov = np.nansum( w * (pred - predbar) * (true - truebar), axis = 0) # weighted covariance between predicted and true
    
    weighted_stdpred = np.sqrt(np.nansum(w * (pred - predbar)**2), axis = 0) # weighted standard deviation predited
    weighted_stdtrue = np.sqrt(np.nansum(w * (true - truebar)**2), axis = 0) # weighted standard deviation true

    correlation = weighted_cov / (weighted_stdpred * weighted_stdtrue)

    return correlation

In [12]:
def skill(pred, true, epsilon = 1e-4):
    # NOTE excluding epsilon = 1e-4 from denominator for now

    mse = np.nanmean((true - pred)**2, axis = 0) # mean square error
    # NOTE above is not equivalent to np.nanvar(true-pred), which excludes bias term
    # MSE = E[(y-x)^2]
    # = (E[y-x])^2 + Var(y-x)
    # = bias^2 + Var(y-x)
    # Can prove the above

    truebar = np.nanmean(true, axis = 0) # mean true

    vartrue = np.nanmean((true - truebar)**2, axis = 0) # variance in true
    # NOTE above is equivalent to np.nanvar()

    skill = 1 - mse / vartrue

    return skill

In [13]:
def weighted_skill(pred, true, r, epsilon = 1e-4):
    # NOTE excluding epsilon = 1e-4 from denominator for now
    # NOTE including epsilon = 1e-4 in the weights in case of uncertainty r ~ 0

    w = 1 / (r + epsilon)

    mse = np.nanmean(( w * (true - pred))**2, axis = 0) # mean square error
    # NOTE above is not equivalent to np.nanvar(true-pred), which excludes bias term

    truebar = np.nanmean(true, axis = 0) # mean true

    vartrue = np.nanmean(( w * (true - truebar))**2, axis = 0) # variance in true
    # NOTE above is equivalent to np.nanvar()

    weighted_skill = 1 - mse / vartrue

    return weighted_skill

In [15]:
uskill = skill(upred, utrue)
vskill = skill(vpred, vtrue)

/tmp/ipykernel_1231525/1496635322.py:16: RuntimeWarning: divide by zero encountered in divide
  skill = 1 - mse / vartrue
/tmp/ipykernel_1231525/1496635322.py:16: RuntimeWarning: invalid value encountered in divide
  skill = 1 - mse / vartrue


In [18]:
# NOTE include timstamp for running bash on model training and eval in the future

# set model for title
MODEL = "Weighted CNN V2"

# set latitude bounds
LAT_MIN = -90
LAT_MAX = -65

cmap = cmo.cm.balance
vmin = -1
vmax = 1

udata = uskill
vdata = vskill

fig, axs = plt.subplots(nrows = 1, ncols = 2, figsize = (12, 3),
                         projection=ccrs.SouthPolarStereo(), constrained_layout = True)

axs.set_extent([-180, 180, LAT_MIN, LAT_MAX])

upcm = axs[0,0].pcolormesh(lon, lat, udata, transform = ccrs.PlateCarree(), cmap = cmap, vimin = vmin ,vmax = vmax)
axs[0,0].coastlines()
plt.colorbar(upcm, ax = axs[0,0], orientation = 'vertical')
axs[0,0].set_title("zonal")

vpcm = axs[0,1].pcolormesh(lon, lat, vdata, transform = ccrs.PlateCarree(), cmap = cmap, vimin = vmin ,vmax = vmax)
axs[0,1].coastlines()
plt.colorbar(vpcm, ax = axs[0,1], orientation = 'vertical')
axs[0,0].set_title("meridional")

fig.suptitle("Skill", fontsize = 16, fontweight = 'bold')

fig.tight_layout()
plt.show()


AttributeError: Figure.set() got an unexpected keyword argument 'projection'